In [ ]:
from cloudyfsps import outObj as ob
from cloudyfsps.outObj import getColors, nColors
from cloudyfsps.generalTools import calcQ
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rc('font', size=16, family='serif', serif=[r'cmr10'], style='normal', variant='normal', stretch='normal', weight='heavy')
mpl.rc('legend', labelspacing=0.1, handlelength=2, fontsize=12)
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

# Figure 15

In [ ]:
import seaborn.apionly as sns
mcmap = sns.diverging_palette(250, 10, s=75, l=40,n=10, center="dark", as_cmap=True)

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputCKC/'
mod_prefix='ZAU'
zau = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
def plt_var(zau, fidmod, val='age', colarr=None, ax=None,
            markersize=14, cname=None, minv=0.0, maxv=1.0):
    if val == 'age':
        modlist = [mod for mod in zau.mods if (mod.logZ == fidmod.logZ) & (mod.logU == fidmod.logU) & (mod.age <= 5.e6)]
        #modlist = [mod for mod in zau.mods if (mod.logZ == fidmod.logZ) & (mod.logU == fidmod.logU)]
    if val == 'logZ':
        modlist = [mod for mod in zau.mods if (mod.age == fidmod.age) & (mod.logU == fidmod.logU) & (mod.logZ >= -1.0)]
    if val == 'logU':
        modlist = [mod for mod in zau.mods if (mod.age == fidmod.age) & (mod.logZ == fidmod.logZ)]
    xarr = np.array([mod.log_NIIb_Ha for mod in modlist])
    yarr = np.array([mod.log_OIIIb_Hb for mod in modlist])
    if colarr is None:
        if val == 'age':
            colarr = np.array([mod.age/1.e6 for mod in modlist])
            fidcol = fidmod.age/1.e6
            print colarr, fidcol
        else:
            colarr = np.array([mod.__getattribute__(val) for mod in modlist])
            fidcol = fidmod.__getattribute__(val)
    if cname is None:
        sM = getColors(colarr, cmap=mcmap, minv=0.0, maxv=1.0)
    else:
        sM = getColors(colarr, cname=cname, minv=minv, maxv=maxv)
    if ax is not None:
        ax.plot(xarr, yarr, lw=1, color='k')
        for x,y,c in zip(xarr,yarr,colarr):
            marker='o'
            color=sM.to_rgba(c)
            markersz=markersize
            if c == fidcol:
                marker='*'
                color='k'
                markersz=20
            ax.plot(x,y, marker=marker,
                    markersize=markersz,
                    color=color, linestyle='None',
                    markeredgecolor='None', label=r'${0:.1f}$'.format(c))
        return ax
    else:
        return xarr, yarr, colarr

In [ ]:
fid = dict(logR=19., nH=100., age=1.0e6, logZ=-0.1, logU=-2.5)
fidmod1 = [mod for mod in zau.mods if ((mod.logZ == fid['logZ']) & (mod.age == fid['age']) & (mod.logU == fid['logU']))][0]

fid = dict(logR=19., nH=100., age=2.0e6, logZ=-0.1, logU=-2.5)
fidmod2 = [mod for mod in zau.mods if ((mod.logZ == fid['logZ']) & (mod.age == fid['age']) & (mod.logU == fid['logU']))][0]

fid = dict(logR=19., nH=100., age=1.0e6, logZ=-0.1, logU=-2.5)
fidmod3 = [mod for mod in zau.mods if ((mod.logZ == fid['logZ']) & (mod.age == fid['age']) & (mod.logU == fid['logU']))][0]

fid = dict(logR=19., nH=100., age=2.0e6, logZ=-0.1, logU=-2.5)
fidmod4 = [mod for mod in zau.mods if ((mod.logZ == fid['logZ']) & (mod.age == fid['age']) & (mod.logU == fid['logU']))][0]


In [ ]:
sxname = 'logNHa'
syname = 'log_OIII_Hb'
xlim=[-1.7, 0.1]
ylim=[-2.4, 1.2]
import pickle
from numpy.lib import recfunctions

samplefile = open('../SNcuts_kappa20_sample.pickle', 'rb')
sample = pickle.load(samplefile)
samplefile.close()

sample = recfunctions.append_fields(sample, ['log_OIII_Hb','log_SII_Ha'],
                                    [np.log10(sample['OIII5007']/sample['Hbeta']),
                                    np.log10((sample['SII6717']+sample['SII6731'])/sample['Halpha'])],
                                    asrecarray=True)

In [ ]:
from matplotlib.colors import LogNorm
def get_contours(x, y, bins=(50, 50), ranges=None):
    if ranges:
        H, xedges, yedges = np.histogram2d(x, y, bins=bins, range=ranges)
    else:
        H, xedges, yedges = np.histogram2d(x, y, bins=bins)
    xmid = (xedges[1:] + xedges[:-1]) / 2.0
    ymid = (yedges[1:] + yedges[:-1]) / 2.0
    return xmid, ymid, H.T
def plot_2Dhist(x, y, xlabel=None, ylabel=None, cblabel=None, ranges=[[-0.007, 0.002],[-0.014, 0.005]],
                vmin=1, vmax=10**5, normed=False, plot_cb=True,
                filename=None, annotate_string=None, annotate_loc=None, ax=None, cmap=plt.cm.viridis):
    xmid, ymid, H = get_contours(x, y, bins=(50, 50), ranges=ranges)
    
    if normed == True:
        norm = np.sum(H, axis=0)  # get total number of galaxies in each mass bin
        # hacky way to make sure division is done correctly
        H_norm = np.zeros(np.shape(H))
        for col in range(np.shape(H)[1]):
            H_norm[:, col] = H[:, col] / norm[col]
    else:
        H_norm = H
    if ax is None:
        fig, ax = plt.subplots(figsize=(6.5, 5))
    #plt.gcf().subplots_adjust(bottom=0.15)
    # LogNorm was 0.001, 0.5
    ax.imshow(H_norm, origin='lower', aspect='auto',
              interpolation='nearest', cmap=cmap, norm=LogNorm(vmin=vmin, vmax=vmax),
              extent=(ranges[0][0], ranges[0][1], ranges[1][0], ranges[1][1]), vmin=vmin,
              vmax=vmax)  # cmap=plt.cm.binary, cmap=plt.cm.cubehelix_r

    if xlabel:
        plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)
    if plot_cb:
        cb = plt.colorbar()
    if cblabel:
        cb.set_label(cblabel)
    
    if annotate_string:
        plt.annotate(annotate_string, annotate_loc, fontsize=16)
    return


In [ ]:
fig, axes = plt.subplots(1,2, figsize=(9,4), sharex=True, sharey=True)
fidmods = [fidmod1, fidmod2, fidmod3, fidmod4]
fidmods = [fidmod1, fidmod1, fidmod4, fidmod4]

labs = [r'1 Myr', r'1 Myr', r'2 Myr', r'2 Myr']
        #r'\textbf{{2 Myr, log Z/Z$_{{\odot}}$ = {}}}'.format(fidmods[1].logZ),
        #r'\textbf{{1 Myr, log $\mathcal{{U}}_0$ = {0:.1f}}}'.format(fidmods[1].logU)]#,
        #r'\textbf{{2 Myr, log $\mathcal{{U}}_0$ = {0:.1f}}}'.format(fidmods[3].logU)]
titles = [r'$\log \mathcal{U}_0$', #r'$\log \mathcal{U}_0$',
          r'log Z/Z$_{\odot}$',
          r'$\log \mathcal{U}_0$', #r'$\log \mathcal{U}_0$',
          r'log Z/Z$_{\odot}$']#, r'\textbf{log Z/Z$_{\odot}$}']
vals = ['logU', #'logU',
        'logZ',
        'logU',
        'logZ']#, 'logZ']
cnames = [None,# None,
          'inferno_r', None, 'inferno_r']#, 'inferno_r']

ii = range(4)
i=0
for ax, val, cname, lab, title, fidmod in zip([fig.axes[0], fig.axes[1], fig.axes[0], fig.axes[1]], vals, cnames, labs, titles, fidmods):
    #ax = fig.axes[i]
    if i < 2:
        plot_2Dhist(sample[sxname], sample[syname], ranges=[[-2.0,0.0], ylim], ax=ax,cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
    #    zau.makeBPT(data_only=True, ax=ax, plt_pars=dict(lab='__nolegend__'))
    plt_var(zau, fidmod, ax=ax, val=val, cname=cname, minv=0.1,maxv=0.9)
    if i < 2:
        leg = ax.legend(numpoints=1, loc=3, frameon=False, fontsize=14, title=title)
        plt.setp(leg.get_title(), fontsize=16)
    i+=1
ax.set_xlim(-1.9,0.4)
ax.set_ylim(-1.6, 1.1)
fig.axes[0].annotate('1 Myr', xy=(0.02, 0.85), xycoords='axes fraction', size=20, ha='left')
fig.axes[0].annotate('2 Myr', xy=(0.05, 0.75), xycoords='axes fraction', size=20, ha='left')

fig.axes[1].annotate('1 Myr', xy=(0.55, 0.75), xycoords='axes fraction', size=20, ha='left')
fig.axes[1].annotate('2 Myr', xy=(0.65, 0.65), xycoords='axes fraction', size=20, ha='left')

fig.axes[0].set_xlabel(r'log [N II]$\;\lambda$6584 / H$\alpha$', size=16)
fig.axes[1].set_xlabel(r'log [N II]$\;\lambda$6584 / H$\alpha$', size=16)
fig.axes[0].set_ylabel(r'log [O III]$\;\lambda$5007 / H$\beta$', size=16)
fig.subplots_adjust(hspace=0, wspace=0, left=0.08, right=0.99, bottom=0.14, top=0.98)
fig.savefig('./f15.pdf', dpi=300)

# Figure 16

In [ ]:
from vz import plot_bpt as vzBPT
import pickle
from numpy.lib import recfunctions
from cloudyfsps.outObj import add_dopita

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputCKC/'
mod_prefix='ZAU'
zau = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputMIST/'
mod_prefix='ZAU'
mau = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputCSFH/'
mod_prefix='ZAU'
csf = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
samplefile = open('../SNcuts_kappa20_sample.pickle', 'rb')
sample = pickle.load(samplefile)
samplefile.close()
# add fields to recarr
sample = recfunctions.append_fields(sample, ['log_OIII_Hb','log_SII_Ha'],
                                    [np.log10(sample['OIII5007']/sample['Hbeta']),
                                    np.log10((sample['SII6717']+sample['SII6731'])/sample['Halpha'])],
                                    asrecarray=True)

In [ ]:
from matplotlib.colors import LogNorm
def get_contours(x, y, bins=(50, 50), ranges=None):
    if ranges:
        H, xedges, yedges = np.histogram2d(x, y, bins=bins, range=ranges)
    else:
        H, xedges, yedges = np.histogram2d(x, y, bins=bins)
    xmid = (xedges[1:] + xedges[:-1]) / 2.0
    ymid = (yedges[1:] + yedges[:-1]) / 2.0
    return xmid, ymid, H.T
def plot_2Dhist(x, y, xlabel=None, ylabel=None, cblabel=None, ranges=[[-0.007, 0.002],[-0.014, 0.005]],
                vmin=1, vmax=10**5, normed=False, plot_cb=True,
                filename=None, annotate_string=None, annotate_loc=None, ax=None, cmap=plt.cm.viridis):
    xmid, ymid, H = get_contours(x, y, bins=(50, 50), ranges=ranges)
    
    if normed == True:
        norm = np.sum(H, axis=0)  # get total number of galaxies in each mass bin
        # hacky way to make sure division is done correctly
        H_norm = np.zeros(np.shape(H))
        for col in range(np.shape(H)[1]):
            H_norm[:, col] = H[:, col] / norm[col]
    else:
        H_norm = H
    if ax is None:
        fig, ax = plt.subplots(figsize=(6.5, 5))
    #plt.gcf().subplots_adjust(bottom=0.15)
    # LogNorm was 0.001, 0.5
    ax.imshow(H_norm, origin='lower', aspect='auto',
              interpolation='nearest', cmap=cmap, norm=LogNorm(vmin=vmin, vmax=vmax),
              extent=(ranges[0][0], ranges[0][1], ranges[1][0], ranges[1][1]), vmin=vmin,
              vmax=vmax)  # cmap=plt.cm.binary, cmap=plt.cm.cubehelix_r

    if xlabel:
        plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)
    if plot_cb:
        cb = plt.colorbar()
    if cblabel:
        cb.set_label(cblabel)
    
    if annotate_string:
        plt.annotate(annotate_string, annotate_loc, fontsize=16)
    return


In [ ]:
def altBPT(zau, age=2.82e9, ax=None, xname='log_NIIb_Ha', yname='log_OIIIb_Hb', do_label=False,
           grid_xname='logU', grid_yname='logZ', xlim=(-2.0, 1.0), ylim=(-2.0, 1.0),
           xcname='Blues', ycname='inferno', lw=4, add_starlab=True,alpha=1.0,
           xclims=(-4.0, -1.0), yclims=(-2.0, 0.5), xv=(0.2, 1.0), yv=(0.2, 0.9), add_star=True):
    if ax is None:
        fig, ax = plt.subplots(1)
    use_mods = [mod for mod in zau.mods if ((mod.age == age) and \
                                            (mod.__getattribute__(grid_xname) >= xclims[0]) and \
                                            (mod.__getattribute__(grid_xname) <= xclims[1]) and \
                                            (mod.__getattribute__(grid_yname) >= yclims[0]) and \
                                            (mod.__getattribute__(grid_yname) <= yclims[1]))]
    grid_x = np.array([x for x in zau.__getattribute__(grid_xname+'_vals') if (x >= xclims[0]) & (x <= xclims[1])])
    grid_y = np.array([y for y in zau.__getattribute__(grid_yname+'_vals') if (y >= yclims[0]) & (y <= yclims[1])])
    
    sMx = getColors(grid_x, cname=xcname, maxv=xv[1], minv=xv[0])
    sMy = getColors(grid_y, cname=ycname, maxv=yv[1], minv=yv[0])
    FLxx = []
    FLxy = []
    for gx in grid_x:
        mods = [mod for mod in use_mods if (mod.__getattribute__(grid_xname) == gx)]
        x_x = np.array([mod.__getattribute__(xname) for mod in mods])
        x_y = np.array([mod.__getattribute__(yname) for mod in mods])
        ax.plot(x_x, x_y, lw=lw, color=sMx.to_rgba(gx), markeredgecolor='None', alpha=alpha)
        if gx == grid_x[0] or gx == grid_x[-1]:
            FLxx.append(x_x)
            FLxy.append(x_y)
    FLyx = []
    FLyy = []
    for gy in grid_y:
        mods = [mod for mod in use_mods if (mod.__getattribute__(grid_yname) == gy)]
        y_x = np.array([mod.__getattribute__(xname) for mod in mods])
        y_y = np.array([mod.__getattribute__(yname) for mod in mods])
        ax.plot(y_x, y_y, lw=lw, color=sMy.to_rgba(gy), markeredgecolor='None', alpha=alpha)
        if gy == grid_y[0] or gy == grid_y[-1]:
            FLyx.append(y_x)
            FLyy.append(y_y)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    if add_star:
        x1,y1 = FLxx[1][0], FLxy[1][0]
        ax.plot(x1, y1, '*', markersize=18,
                color=sMy.to_rgba(grid_y[0]),
                markeredgecolor='None', zorder=15)
        if add_starlab:
            ax.annotate(r'$\log\;$Z$/$Z$_{{\odot}}={0:.1f}$'.format(grid_y[0]),
                        xy=(x1,y1), xycoords='data',
                        xytext=(15,10), textcoords='offset points',
                        ha='left', va='top', size=14)
        x2,y2=FLyx[1][0], FLyy[1][0]
        ax.plot(x2, y2, '*', markersize=22, color=sMy.to_rgba(grid_y[-1]),
                markeredgecolor='None', zorder=15)
        if add_starlab:
            ax.annotate(r'$\log\;$Z$/$Z$_{{\odot}}={0:.1f}$'.format(grid_y[-1]),
                        xy=(x2, y2), xycoords='data',
                        xytext=(-10,0), textcoords='offset points',
                        ha='right', va='top', size=14)
    return

In [ ]:
sxname = 'logNHa'
syname = 'log_OIII_Hb'
xlim=[-1.7, 0.1]
ylim=[-2.4, 1.2]

ratio='NIIb'
age = 1.0e6
xlab=r'$\log$ [N II]$\;\lambda$6584 / H$\alpha$'
ylab=r'$\log$ [O III]$\;\lambda$5007 / H$\beta$'
xname, yname = 'log_NIIb_Ha', 'log_OIIIb_Hb'

ages = [1.0e6, 3.0e6, 5.0e6]
fig, axes = plt.subplots(2,3, figsize=(9, 6))
for age, ax in zip(ages, fig.axes[0:4]):
    add_starlab=False
    if age == 5.0e6:
        add_starlab=True
    plot_2Dhist(sample[sxname], sample[syname], ranges=[[-2.0,0.0], ylim], ax=ax,
                cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
    NII = np.linspace(-4.5, 0.04)
    OIII = 0.61/(NII - 0.05) + 1.3
    ax.plot(NII, OIII, '--', color='k', lw=2)
    NII = np.linspace(-4.5, 0.46, 100)
    OIII = 0.61/(NII - 0.47) + 1.19
    ax.plot(NII, OIII, color='k', lw=2)
    vzBPT(True, ax=ax, line_ratio=ratio)
    altBPT(zau, age=age, ax=ax, xname=xname, yname=yname,
           yclims=(-1.0, 0.3), ycname='Blues', add_starlab=add_starlab,
           xlim=xlim, ylim=ylim,xv=(1.0, 1.0), yv=(1.0, 1.0), lw=2)
    lab = "{0:.0f} Myr".format(age/1.e6)
    ax.annotate(lab, xy=(0.95, 0.95), xycoords='axes fraction', ha='right', va='top', size=20)
    ax.set_xlabel(' ')
    ax.set_ylabel(' ')
fig.axes[0].annotate("Burst", xy=(0.02, 0.02),
                     xycoords='axes fraction', ha='left', va='bottom', size=24)
for age, ax in zip(ages, fig.axes[3::]):
    add_starlab=False
    if age == 5.0e6:
        add_starlab=True
    plot_2Dhist(sample[sxname], sample[syname], ranges=[[-2.0,0.0], ylim], ax=ax,
                cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
    NII = np.linspace(-4.5, 0.04)
    OIII = 0.61/(NII - 0.05) + 1.3
    ax.plot(NII, OIII, '--', color='k', lw=2)
    NII = np.linspace(-4.5, 0.46, 100)
    OIII = 0.61/(NII - 0.47) + 1.19
    ax.plot(NII, OIII, color='k', lw=2)
    vzBPT(True, ax=ax, line_ratio=ratio)
    altBPT(csf, age=age, ax=ax, xname=xname, yname=yname,
           yclims=(-1.0, 0.3), ycname='Greens', xcname='Greens', add_starlab=add_starlab,
           xlim=xlim, ylim=ylim,xv=(1.0, 1.0), yv=(1.0, 1.0), lw=2)
    lab = "{0:.0f} Myr".format(age/1.e6)
    ax.annotate(lab, xy=(0.95, 0.95), xycoords='axes fraction', ha='right', va='top', size=20)
    ax.set_xlabel(' ')
    ax.set_ylabel(' ')
fig.axes[3].annotate("CSFR", xy=(0.02, 0.02), xycoords='axes fraction',
                     ha='left', va='bottom', size=24)
plt.setp([a.get_xticklabels() for a in fig.axes[0:2]], visible=False);
plt.setp(fig.axes[1].get_yticklabels(), visible=False);
plt.setp(fig.axes[3].get_yticklabels(), visible=False);
plt.setp([a.get_yticklabels() for a in fig.axes], visible=False);
plt.setp([a.get_yticklabels() for a in [fig.axes[0], fig.axes[3]]], visible=True);

fig.subplots_adjust(left=0.08, wspace=0, hspace=0, right=0.98, top=0.98, bottom=0.08)
#fig.axes[2].set_xlabel(xlab, size=14)
fig.axes[3].set_xlabel(xlab, size=14)
fig.axes[4].set_xlabel(xlab, size=14)
fig.axes[5].set_xlabel(xlab, size=14)
fig.axes[0].set_ylabel(ylab, size=14)
fig.axes[3].set_ylabel(ylab, size=14)
fig.savefig('./f16.pdf', dpi=300)

# Figure 17

In [ ]:
sxname = 'logNHa'
syname = 'log_OIII_Hb'
xlim=[-1.7, 0.1]
ylim=[-2.4, 1.4]

ratio='NIIb'
xlab=r'$\log$ [N II]$\;\lambda$6584 / H$\alpha$'
ylab=r'$\log$ [O III]$\;\lambda$5007 / H$\beta$'
xname, yname = 'log_NIIb_Ha', 'log_OIIIb_Hb'
age=4.0e6

fig, ax = plt.subplots(1, figsize=(6.5,5.5))
plot_2Dhist(sample[sxname], sample[syname], ranges=[[-2.0,0.0], ylim], ax=ax, cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
NII = np.linspace(-4.5, 0.04)
OIII = 0.61/(NII - 0.05) + 1.3
ax.plot(NII, OIII, '--', color='k', lw=2)

NII = np.linspace(-4.5, 0.46, 100)
OIII = 0.61/(NII - 0.47) + 1.19
ax.plot(NII, OIII, color='k', lw=2)
add_dopita(ax=ax, add_labels=False, lw=2, color='slategray',logq_val=str('all'),
           kappa_val=np.inf, line_ratio='NIIb', zlims=(-1.4, 0.8))

altBPT(csf, age=age, ax=ax, xname=xname, yname=yname, alpha=1.0,add_star=True, add_starlab=True,
       yclims=(-1.0, 0.3), #ycname='Greens', xcname='Greens', add_starlab=False,
       xlim=xlim, ylim=ylim)#, xv=(0.8, 0.8), yv=(0.8, 0.8), lw=2)

ax.set_xlabel(xlab, size=16)
ax.set_ylabel(ylab, size=16)
fig.subplots_adjust(top=0.98, right=0.98, left=0.12, bottom=0.12, wspace=0, hspace=0)
ax.set_xlim(-2.6, -0.1)
ax.set_ylim(-2.6, 1.4)
ax.annotate(r'4 Myr CSFR', xy=(0.02, 0.02), xycoords='axes fraction', size=22, ha='left', va='bottom')
#ax.annotate(r'4 Myr Constant SF', xy=(0.4, 0.82), xycoords='axes fraction',
#            fontsize=22, xytext=(0.88,0.95), textcoords='axes fraction', ha='right',
#            arrowprops=dict(arrowstyle='wedge', connectionstyle='arc3, rad=0.3', fc='Green', ec='None'))
ax.annotate(r'Dopita+2013 ($\kappa = \infty$)', xy=(0.02, 0.68), xycoords='axes fraction', fontsize=22,
            xytext=(0.2,0.98), textcoords='axes fraction', ha='left', va='top',
            arrowprops=dict(arrowstyle='wedge', connectionstyle='arc3, rad=0.5', fc='slategray', ec='None'))
fig.subplots_adjust(left=0.12, right=0.98, top=0.98, bottom=0.12)
fig.savefig('./f17.pdf', dpi=300)

# Figure 18

In [ ]:
sxname = 'NII/OII'
syname = 'OIII/OII'
ylim= [-2.3, 1.9]
xlim=[-1.6, 0.6]
ratio='OII'
age = 1.0e6
xlab, ylab = r'log [N II]/[O II]', r'log [O III]/[O II]'
xname, yname = 'log_NII_OII', 'log_OIII_OII'

fig, axes = plt.subplots(2, figsize=(4.5, 7.), sharex=True, sharey=True)
#fig, axes = plt.subplots(2, figsize=(3.5, 6))

ax = fig.axes[0]
add_starlab=True

plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax, 
            cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(zau, age=age, ax=ax, xname=xname, yname=yname,
       yclims=(-1.0, 0.2),lw=3,
       xlim=xlim, ylim=ylim, add_starlab=add_starlab)
lab = "{0:.0f} Myr Burst".format(age/1.e6)
#ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
ax.annotate(lab, xy=(0.02, 0.02),xycoords='axes fraction', ha='left', va='bottom', size=20)

ax = fig.axes[1]
age = 4.0e6
plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax,
            cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(csf, age=age, ax=ax, xname=xname, yname=yname,
       yclims=(-1.0, 0.2), 
       xlim=xlim, ylim=ylim, lw=3, add_starlab=add_starlab)
lab = "{0:.0f} Myr CSFR".format(age/1.e6)
#ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
ax.annotate(lab, xy=(0.02, 0.02), xycoords='axes fraction', ha='left', va='bottom', size=20)

plt.setp(fig.axes[0].get_xticklabels(), visible=False);
#plt.setp(fig.axes[1].get_yticklabels(), visible=False);
#plt.setp(fig.axes[3].get_yticklabels(), visible=False);
#plt.setp([a.get_yticklabels() for a in [fig.axes[1], fig.axes[3]]], visible=False);
fig.subplots_adjust(left=0.00, wspace=0, hspace=0)
fig.axes[1].set_xlabel(xlab, size=14)
#fig.axes[3].set_xlabel(xlab, size=14)
fig.axes[0].set_ylabel(ylab, size=14)
fig.axes[1].set_ylabel(ylab, size=14)

fig.subplots_adjust(top=0.98, right=0.98, left=0.15, bottom=0.08, wspace=0, hspace=0)
fig.savefig('./finalFigs/f18.pdf', dpi=300)

# Figure 19

In [ ]:
[mod.__setattr__('log_NeIII_OII', np.log10(mod.NeIIIb/mod.OIIb)) for mod in zau.mods];
zau.add_arrs('log_NeIII_OII')
[mod.__setattr__('log_NeIII_OII', np.log10(mod.NeIIIb/mod.OIIb)) for mod in csf.mods];
csf.add_arrs('log_NeIII_OII')

In [ ]:
sxname = 'NII/OII'
syname = 'OIII/OII'
ylim= [-1.1, 1.1]
xlim=[-1.7, 0.1]
ratio='NeIII'
age = 1.0e6
xlab, ylab = r'log [Ne III]/[O II]', r'log [O III]/[O II]'
xname, yname = 'log_NeIII_OII', 'log_OIII_OII'
fig, axes = plt.subplots(2, figsize=(4.5, 7.), sharex=True, sharey=True)
ax=axes[0]
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(zau, age=age, ax=ax, xname=xname, yname=yname,
       yclims=(-1.0, 0.2),lw=3,
       xlim=xlim, ylim=ylim, add_starlab=add_starlab)
lab = "{0:.0f} Myr Burst".format(age/1.e6)
ax.annotate(lab, xy=(0.02, 0.98), xycoords='axes fraction', ha='left', va='top', size=20)
#ax.set_xlabel(' ')
ax.set_ylabel(ylab, size=14)
#fig.axes[0].annotate("Burst", xy=(0.02, 0.02),
#                     xycoords='axes fraction', ha='left', va='bottom', size=24)
age=4.0e6
ax=axes[1]
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(csf, age=age, ax=ax, xname=xname, yname=yname,
       yclims=(-1.0, 0.2), 
       xlim=xlim, ylim=ylim, lw=3, add_starlab=add_starlab)
lab = "{0:.0f} Myr CSFR".format(age/1.e6)
ax.annotate(lab, xy=(0.02, 0.98), xycoords='axes fraction', ha='left', va='top', size=20)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
#fig.axes[2].annotate("CSFR", xy=(0.02, 0.02), xycoords='axes fraction',
#                     ha='left', va='bottom', size=24)
plt.setp(fig.axes[0].get_xticklabels(), visible=False);
#fig.axes[2].set_xlabel(xlab, size=14)
ax.set_xlabel(xlab, size=14)
ax.set_ylabel(ylab, size=14)
#fig.axes[2].set_ylabel(ylab, size=14)
fig.subplots_adjust(top=0.98, right=0.98, left=0.15, bottom=0.08, wspace=0, hspace=0)
fig.savefig('./f19.pdf', dpi=300)

# Figure 20

In [ ]:

sxname = 'logR23'
syname = 'OIII/OII' 
ylim= [-2.2, 1.8]
xlim=[-0.8, 1.2]
ratio='R23'
age = 1.0e6
xlab = r'log [O II] + [O III] / H$\beta$ (R23)'
ylab= r'log [O III] / [O II]'
xname, yname = 'R23', 'log_OIII_OII'
add_starlab=False
fig, axes = plt.subplots(2, figsize=(4.5, 7.), sharex=True, sharey=True)
ax=axes[0]
plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax, cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(zau, age=age, ax=ax, xname=xname, yname=yname, add_starlab=add_starlab,
       yclims=(-2.0, 0.2),
       xlim=xlim, ylim=ylim)
lab = "{0:.0f} Myr Burst".format(age/1.e6)
ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)
ax.set_ylabel(ylab, size=14)


age=4.0e6
ax=axes[1]
plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax, cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(csf, age=age, ax=ax, xname=xname, yname=yname,
       yclims=(-2.0, 0.2), 
       xlim=xlim, ylim=ylim, lw=3, add_starlab=add_starlab)
lab = "{0:.0f} Myr CSFR".format(age/1.e6)
ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)

plt.setp(fig.axes[0].get_xticklabels(), visible=False);

#fig.axes[2].set_xlabel(xlab, size=14)
ax.set_xlabel(xlab, size=14)
ax.set_ylabel(ylab, size=14)
#fig.axes[2].set_ylabel(ylab, size=14)


fig.subplots_adjust(top=0.98, right=0.98, left=0.15, bottom=0.08, wspace=0, hspace=0)
fig.savefig('./f20.pdf'.format(xname, yname, age/1.e6), dpi=300)

# Figure 21

In [ ]:
sxname = 'log_SII_Ha' #SII Ha
syname = 'log_OIII_Hb' #Oiiiha
ylim= [-2.1, 1.3]
xlim=[-2.1, 0.1]
ratio='SII'
age = 1.0e6
xlab, ylab = r'log [S II]$\lambda$6716,6731 / H$\alpha$', r'log [O III]$\lambda$5007 / H$\beta$'
xname, yname = sxname, syname
add_starlab=False
fig, axes = plt.subplots(2, figsize=(4.5, 7.), sharex=True, sharey=True)
ax=axes[0]
plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax, cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(zau, age=age, ax=ax, xname=xname, yname=yname, add_starlab=add_starlab,
       yclims=(-2.0, 0.2),
       xlim=xlim, ylim=ylim)
SII = np.linspace(-3.0, 0.31)
OIII = 0.72/(SII - 0.32) + 1.3
ax.plot(SII, OIII, color='k', lw=2)
lab = "{0:.0f} Myr Burst".format(age/1.e6)
ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)
ax.set_xlabel(xlab, size=16)
ax.set_ylabel(ylab, size=16)




age=4.0e6
ax=axes[1]
plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax, cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(csf, age=age, ax=ax, xname=xname, yname=yname,
       yclims=(-2.0, 0.2), 
       xlim=xlim, ylim=ylim, lw=3, add_starlab=add_starlab)
lab = "{0:.0f} Myr CSFR".format(age/1.e6)
ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)

plt.setp(fig.axes[0].get_xticklabels(), visible=False);
fig.subplots_adjust(top=0.98, right=0.98, left=0.15, bottom=0.08, wspace=0, hspace=0)
#fig.axes[2].set_xlabel(xlab, size=14)
SII = np.linspace(-3.0, 0.31)
OIII = 0.72/(SII - 0.32) + 1.3
ax.plot(SII, OIII, color='k', lw=2)
ax.set_xlabel(xlab, size=14)
ax.set_ylabel(ylab, size=14)
#fig.axes[2].set_ylabel(ylab, size=14)

fig.savefig('./f21.pdf'.format(xname, yname, age/1.e6), dpi=300)